In [1]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import logging
import copy
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import kneighbors_graph
import torch.distributions.bernoulli as bernoulli

from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torch.distributions.bernoulli as bernoulli

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

### Transfer Source Data & Model (Concare)

In [2]:
data_path = './data/Challenge/normalized/'
small_part = False
arg_timestep = 1.0
batch_size = 256
epochs = 100

In [3]:
# all_x = pickle.load(open(data_path + 'x.dat', 'rb'))
# all_y = pickle.load(open(data_path + 'y.dat', 'rb'))
# all_names = pickle.load(open(data_path + 'name.dat', 'rb'))
# # time = pickle.load(open(data_path + 'time', 'rb'))
# # weight = pickle.load(open(data_path + 'weight', 'rb'))
# static = pickle.load(open(data_path + 'demo.dat', 'rb'))
# mask_x = pickle.load(open(data_path + 'mask_x.dat', 'rb'))
# mask_demo = pickle.load(open(data_path + 'mask_demo.dat', 'rb'))
# all_x_len = [len(i) for i in all_x]

# print(all_x[0])
# print(mask_x[0])
# print(all_names[0])
# print(static[0])

In [4]:
train_x = pickle.load(open(data_path + 'train_x.dat', 'rb'))
train_y = pickle.load(open(data_path + 'train_y.dat', 'rb'))
train_names = pickle.load(open(data_path + 'train_names.dat', 'rb'))
train_static = pickle.load(open(data_path + 'train_static.dat', 'rb'))
train_x_len = pickle.load(open(data_path + 'train_x_len.dat', 'rb'))
train_mask_x = pickle.load(open(data_path + 'train_mask_x.dat', 'rb'))

dev_x = pickle.load(open(data_path + 'dev_x.dat', 'rb'))
dev_y = pickle.load(open(data_path + 'dev_y.dat', 'rb'))
dev_names = pickle.load(open(data_path + 'dev_names.dat', 'rb'))
dev_static = pickle.load(open(data_path + 'dev_static.dat', 'rb'))
dev_x_len = pickle.load(open(data_path + 'dev_x_len.dat', 'rb'))
dev_mask_x = pickle.load(open(data_path + 'dev_mask_x.dat', 'rb'))

test_x = pickle.load(open(data_path + 'test_x.dat', 'rb'))
test_y = pickle.load(open(data_path + 'test_y.dat', 'rb'))
test_names = pickle.load(open(data_path + 'test_names.dat', 'rb'))
test_static = pickle.load(open(data_path + 'test_static.dat', 'rb'))
test_x_len = pickle.load(open(data_path + 'test_x_len.dat', 'rb'))
test_mask_x = pickle.load(open(data_path + 'test_mask_x.dat', 'rb'))

print(len(train_x))
print(len(dev_x))
print(len(test_x))
print(sum(train_y)/len(train_y))
print(sum(dev_y)/len(dev_y))
print(sum(test_y)/len(test_y))

32268
4033
4035
0.07267261683401513
0.07265063228365981
0.07286245353159851


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
# device = torch.device('cuda')
print("available device: {}".format(device))

available device: cuda:0


In [6]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [7]:
def get_re_loss(y_pred, y_true):
    loss = torch.nn.MSELoss()
    return loss(y_pred, y_true)

In [8]:
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))


    return np.array(sents_padded)

In [9]:
def batch_iter(x, y, mask, lens, batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size) # 向下取整
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size] #  fetch out all the induces
        
        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx], mask[idx], lens[idx]))
       
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    
        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
        batch_mask_x = [e[2] for e in examples]
#         batch_name = [e[2] for e in examples]
        batch_lens = [e[3] for e in examples]

        yield batch_x, batch_y, batch_mask_x, batch_lens

In [10]:
def length_to_mask(length, max_len=None, dtype=None):
    """length: B.
    return B x max_len.
    If max_len is None, then max of length will be used.
    """
    assert len(length.shape) == 1, 'Length shape should be 1 dimensional.'
    max_len = max_len or length.max().item()
    mask = torch.arange(max_len, device=length.device,
                        dtype=length.dtype).expand(len(length), max_len) < length.unsqueeze(1)
    if dtype is not None:
        mask = torch.as_tensor(mask, dtype=dtype, device=length.device)
    return mask

In [11]:
class SingleAttention(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', demographic_dim=12, time_aware=False, use_demographic=False):
        super(SingleAttention, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim
        self.use_demographic = use_demographic
        self.demographic_dim = demographic_dim
        self.time_aware = time_aware

        # batch_time = torch.arange(0, batch_mask.size()[1], dtype=torch.float32).reshape(1, batch_mask.size()[1], 1)
        # batch_time = batch_time.repeat(batch_mask.size()[0], 1, 1)
        
        if attention_type == 'add':
            if self.time_aware == True:
                # self.Wx = nn.Parameter(torch.randn(attention_input_dim+1, attention_hidden_dim))
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
                self.Wtime_aware = nn.Parameter(torch.randn(1, attention_hidden_dim))
                nn.init.kaiming_uniform_(self.Wtime_aware, a=math.sqrt(5))
            else:
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wt = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wd = nn.Parameter(torch.randn(demographic_dim, attention_hidden_dim))
            self.bh = nn.Parameter(torch.zeros(attention_hidden_dim,))
            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wd, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wx, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wt, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'mul':
            self.Wa = nn.Parameter(torch.randn(attention_input_dim, attention_input_dim))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'concat':
            if self.time_aware == True:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim+1, attention_hidden_dim))
            else:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))

            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        else:
            raise RuntimeError('Wrong attention type.')
        
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
    
    def forward(self, input, demo=None):
 
        batch_size, time_step, input_dim = input.size() # batch_size * time_step * hidden_dim(i)
        #assert(input_dim == self.input_dim)

        # time_decays = torch.zeros((time_step,time_step)).to(device)# t*t
        # for this_time in range(time_step):
        #     for pre_time in range(time_step):
        #         if pre_time > this_time:
        #             break
        #         time_decays[this_time][pre_time] = torch.tensor(this_time - pre_time, dtype=torch.float32).to(device)
        # b_time_decays = tile(time_decays, 0, batch_size).view(batch_size,time_step,time_step).unsqueeze(-1).to(device)# b t t 1

        time_decays = torch.tensor(range(time_step-1,-1,-1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device)# 1*t*1
        b_time_decays = time_decays.repeat(batch_size,1,1)# b t 1
        
        if self.attention_type == 'add': #B*T*I  @ H*I
            q = torch.matmul(input[:,-1,:], self.Wt)# b h
            q = torch.reshape(q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            if self.time_aware == True:
                # k_input = torch.cat((input, time), dim=-1)
                k = torch.matmul(input, self.Wx)#b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
                time_hidden = torch.matmul(b_time_decays, self.Wtime_aware)#  b t h
            else:
                k = torch.matmul(input, self.Wx)# b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
            if self.use_demographic == True:
                d = torch.matmul(demo, self.Wd) #B*H
                d = torch.reshape(d, (batch_size, 1, self.attention_hidden_dim)) # b 1 h
            h = q + k + self.bh # b t h
            if self.time_aware == True:
                h += time_hidden
            h = self.tanh(h) #B*T*H
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step))# b t
        elif self.attention_type == 'mul':
            e = torch.matmul(input[:,-1,:], self.Wa)#b i
            e = torch.matmul(e.unsqueeze(1), input.permute(0,2,1)).squeeze() + self.ba #b t
        elif self.attention_type == 'concat':
            q = input[:,-1,:].unsqueeze(1).repeat(1,time_step,1)# b t i
            k = input
            c = torch.cat((q, k), dim=-1) #B*T*2I
            if self.time_aware == True:
                c = torch.cat((c, b_time_decays), dim=-1) #B*T*2I+1
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        # e = torch.exp(e - torch.max(e, dim=-1, keepdim=True).values)
        
        # if self.attention_width is not None:
        #     if self.history_only:
        #         lower = torch.arange(0, time_step).to(device) - (self.attention_width - 1)
        #     else:
        #         lower = torch.arange(0, time_step).to(device) - self.attention_width // 2
        #     lower = lower.unsqueeze(-1)
        #     upper = lower + self.attention_width
        #     indices = torch.arange(0, time_step).unsqueeze(0).to(device)
        #     e = e * (lower <= indices).float() * (indices < upper).float()
        
        # s = torch.sum(e, dim=-1, keepdim=True)
        # mask = subsequent_mask(time_step).to(device) # 1 t t 下三角
        # scores = e.masked_fill(mask == 0, -1e9)# b t t 下三角
        a = self.softmax(e) #B*T
        v = torch.matmul(a.unsqueeze(1), input).squeeze() #B*I

        return v, a

class FinalAttentionQKV(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', dropout=None):
        super(FinalAttentionQKV, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim


        self.W_q = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_k = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_v = nn.Linear(attention_input_dim, attention_hidden_dim)

        self.W_out = nn.Linear(attention_hidden_dim, 1)

        self.b_in = nn.Parameter(torch.zeros(1,))
        self.b_out = nn.Parameter(torch.zeros(1,))

        nn.init.kaiming_uniform_(self.W_q.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_k.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_v.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_out.weight, a=math.sqrt(5))

        self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))
        self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
        self.ba = nn.Parameter(torch.zeros(1,))
        
        nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        
        self.dropout = nn.Dropout(p=dropout)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input):
 
        batch_size, time_step, input_dim = input.size() # batch_size * input_dim + 1 * hidden_dim(i)
        input_q = self.W_q(torch.mean(input,1)) # b h
        input_k = self.W_k(input)# b t h
        input_v = self.W_v(input)# b t h

        if self.attention_type == 'add': #B*T*I  @ H*I

            q = torch.reshape(input_q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            h = q + input_k + self.b_in # b t h
            h = self.tanh(h) #B*T*H
            e = self.W_out(h) # b t 1
            e = torch.reshape(e, (batch_size, time_step))# b t

        elif self.attention_type == 'mul':
            q = torch.reshape(input_q, (batch_size, self.attention_hidden_dim, 1)) #B*h 1
            e = torch.matmul(input_k, q).squeeze()#b t
            
        elif self.attention_type == 'concat':
            q = input_q.unsqueeze(1).repeat(1,time_step,1)# b t h
            k = input_k
            c = torch.cat((q, k), dim=-1) #B*T*2I
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        a = self.softmax(e) #B*T
        if self.dropout is not None:
            a = self.dropout(a)
        v = torch.matmul(a.unsqueeze(1), input_v).squeeze() #B*I

        return v, a

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(device)
    return torch.index_select(a, dim, order_index).to(device)

class PositionwiseFeedForward(nn.Module): # new added
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x)))), None

# class PositionwiseFeedForwardConv(nn.Module):

#     def __init__(self, model_dim=512, ffn_dim=2048, dropout=0.0):
#         super(PositionalWiseFeedForward, self).__init__()
#         self.w1 = nn.Conv1d(model_dim, ffn_dim, 1)
#         self.w2 = nn.Conv1d(model_dim, ffn_dim, 1)
#         self.dropout = nn.Dropout(dropout)
#         self.layer_norm = nn.LayerNorm(model_dim)

#     def forward(self, x):
#         output = x.transpose(1, 2)
#         output = self.w2(F.relu(self.w1(output)))
#         output = self.dropout(output.transpose(1, 2))

#         # add residual and norm layer
#         output = self.layer_norm(x + output)
#         return output

class PositionalEncoding(nn.Module): # new added / not use anymore
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=400):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0 # 下三角矩阵

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)# b h t d_k
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k) # b h t t
    if mask is not None:# 1 1 t t
        scores = scores.masked_fill(mask == 0, -1e9)# b h t t 下三角
    p_attn = F.softmax(scores, dim = -1)# b h t t
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn # b h t v (d_k) 
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, self.d_k * self.h), 3)
        self.final_linear = nn.Linear(d_model, d_model)
        self.attn = None
        self.mask = None
        self.dropout = nn.Dropout(p=dropout)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1) # 1 1 t t

        nbatches = query.size(0)# b
        input_dim = query.size(1)# i+1
        feature_dim = query.size(-1)# i+1

        #input size -> # batch_size * d_input * hidden_dim
        
        # d_model => h * d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))] # b num_head d_input d_k
        
        
        querys = []
        keys = []
        values = []
        for i in range(self.h):
            querys.append(query[:,i].unsqueeze(1))
            keys.append(key[:,i].unsqueeze(1))
            values.append(value[:,i].unsqueeze(1))
            
        
       
        if self.training == True:

            x, attn = attention(querys[0], keys[0], values[0], mask=mask, 
                                     dropout=self.dropout)# b num_head d_input d_v (d_k) 


            self.attn = attn
    #         self.mask = mask
            attn1 = torch.mean(attn[:,0],1)
    #         print(attn1.shape) # 256, 153
    #         attn1_p = attn1/torch.sum(attn1,1).unsqueeze(1)

            attn1_p = 0.1*self.sigmoid(lNorm(attn1)*0.01)
    #         print(attn1_p.shape)


            dis_b1 = bernoulli.Bernoulli(attn1_p)
            to_mask1 = 1 - dis_b1.sample().to(device)
            mask1 = to_mask1.unsqueeze(1).repeat(1,input_dim,1).unsqueeze(1)


            tempx, attn = attention(querys[1], keys[1], values[1], mask=mask1, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            self.mask = mask1
            x = torch.cat((x, tempx),1)

            attn2 = attn1 + torch.mean(attn[:,0],1)
    #         attn2_p = attn2/torch.sum(attn2,1).unsqueeze(1)
            attn2_p = 0.1*self.sigmoid(lNorm(attn2)*0.01)

            dis_b2 = bernoulli.Bernoulli(attn2_p)
            to_mask2 = 1 - dis_b2.sample().to(device)
            mask2 = to_mask2.unsqueeze(1).repeat(1,input_dim,1).unsqueeze(1)

            tempx, attn = attention(querys[2], keys[2], values[2], mask=mask2, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            self.mask = torch.cat((self.mask, mask2), 1)
            x = torch.cat((x, tempx),1)

    #         attn3 = torch.mean(attn[:,0],0)
            attn3 = attn2 + torch.mean(attn[:,0],1)
    #         attn3_p = attn3/torch.sum(attn3,1).unsqueeze(1)
            attn3_p = 0.1*self.sigmoid(lNorm(attn3)*0.01)

            dis_b3 = bernoulli.Bernoulli(attn3_p)
            to_mask3 = 1 - dis_b3.sample().to(device)
            mask3 = to_mask3.unsqueeze(1).repeat(1,input_dim,1).unsqueeze(1)


            tempx, attn = attention(querys[3], keys[3], values[3], mask=mask3, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            self.mask = torch.cat((self.mask, mask3), 1)

    #         attn4 = torch.mean(attn[0,0],0)
            x = torch.cat((x, tempx),1)
    
        

        if self.training == False:
# #             print(1)
            
            x, attn = attention(querys[0], keys[0], values[0], mask=None, 
                                     dropout=self.dropout)# b num_head d_input d_v (d_k) 


            self.attn = attn

            tempx, attn = attention(querys[1], keys[1], values[1], mask=None, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            x = torch.cat((x, tempx),1)



            tempx, attn = attention(querys[2], keys[2], values[2], mask=None, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            x = torch.cat((x, tempx),1)




            tempx, attn = attention(querys[3], keys[3], values[3], mask=None, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)

    #         attn4 = torch.mean(attn[0,0],0)
            x = torch.cat((x, tempx),1)
        



      
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)# batch_size * d_input * hidden_dim

        return self.final_linear(x), torch.zeros(1).to(device)

        


class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-7):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
    
def lNorm(x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    eps = 1e-7
    return (x - mean) / (std + eps)

def cov(m, y=None):
    if y is not None:
        m = torch.cat((m, y), dim=0)
    m_exp = torch.mean(m, dim=1)
    x = m - m_exp[:, None]
    cov = 1 / (x.size(1) - 1) * x.mm(x.t())
    return cov

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        returned_value = sublayer(self.norm(x))
        return x + self.dropout(returned_value[0]) , returned_value[1]

class vanilla_transformer_encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(vanilla_transformer_encoder, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
  
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

    def forward(self, input, lens):
        # input shape [batch_size, timestep, feature_dim]
#         demo_main = self.tanh(self.demo_proj_main(demo_input)).unsqueeze(1)# b hidden_dim
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

     
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h

        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)
            

        contexts = self.SublayerConnection(posi_input, lambda x: self.MultiHeadedAttention(posi_input, posi_input, posi_input, None))# # batch_size * d_input * hidden_dim
    
        DeCov_loss = contexts[1]
        contexts = contexts[0]

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))[0]# # batch_size * d_input * hidden_dim
        

        weighted_contexts = self.FinalAttentionQKV(contexts)[0]
        output = self.output(self.dropout(weighted_contexts))# b 1
        output = self.sigmoid(output)
#         print(weighted_contexts.shape)
          
        return output



In [12]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn
    
epochs = 150
batch_size = 256
input_dim = 33
hidden_dim = 32
d_model = 32
MHD_num_head = 4
d_ff = 64
output_dim = 1

model = vanilla_transformer_encoder(input_dim = input_dim, hidden_dim = hidden_dim, d_model = d_model,  MHD_num_head = MHD_num_head , d_ff = d_ff, output_dim = output_dim).to(device)
# input_dim, d_model, d_k, d_v, MHD_num_head, d_ff, output_dim
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [13]:
total_train_loss = []
total_valid_loss = []
global_best = 0
auroc = []
auprc = []
minpse = []
history = []

pad_token = np.zeros(33)
# begin_time = time.time()
best_auroc = 0
best_auprc = 0
best_sum = 0

    
file_name = './model/concares'

for each_epoch in range(epochs):

    epoch_loss = []
    counter_batch = 0
    model.train()  

    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(train_x, train_y, train_mask_x, train_x_len, batch_size, shuffle=True)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        opt = model(batch_x, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))


        model_loss =  BCE_Loss 

        loss = model_loss

        epoch_loss.append(BCE_Loss.cpu().detach().numpy())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
        optimizer.step()

        if step % 20 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))

    epoch_loss = np.mean(epoch_loss)
    total_train_loss.append(epoch_loss)

    #Validation
    y_true = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        valid_loss = []
        valid_true = []
        valid_pred = []
        for batch_x, batch_y, batch_mask_x, batch_lens in batch_iter(dev_x, dev_y, dev_mask_x, dev_x_len, batch_size):
            batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
            batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

            opt = model(batch_x, batch_lens)

            BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))

            valid_loss.append(BCE_Loss.cpu().detach().numpy())

            y_pred += list(opt.cpu().detach().numpy().flatten())
            y_true += list(batch_y.cpu().numpy().flatten())

        valid_loss = np.mean(valid_loss)
        total_valid_loss.append(valid_loss)
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())
        ret = metrics.print_metrics_binary(y_true, y_pred,verbose = 0)
        history.append(ret)
        #print()

        print('Epoch %d: Loss = %.4f Valid loss = %.4f roc = %.4f'%(each_epoch, total_train_loss[-1], total_valid_loss[-1], ret['auroc']))
        metrics.print_metrics_binary(y_true, y_pred)

        cur_auroc = ret['auroc']
        if cur_auroc > best_auroc:
            best_auroc = cur_auroc
#             best_auprc = ret['auprc']
#             best_minpse = ret['minpse']
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name)
            print('------------ Save best model - AUROC: %.4f ------------'%cur_auroc) 
        
        cur_prc = ret['auprc']
        cur_minpse = ret['minpse']
        
        if cur_prc > best_auprc:
            best_auprc = cur_prc
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name+"prc")
            print('\n------------ Save best-prc model ------------\n')

        cur_sum = cur_auroc + cur_prc + cur_minpse
    #     print(cur_sum)
        if cur_sum > best_sum:
            best_sum = cur_sum
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name+"sum")
            print('\n------------ Save best-sum model ------------\n')

print('auroc %.4f'%(best_auroc))
print('auprc %.4f'%(best_auprc))
print('minpse %.4f'%(best_minpse))  

/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 Batch 0: Train Loss = 0.6263
Epoch 0 Batch 20: Train Loss = 0.3647
Epoch 0 Batch 40: Train Loss = 0.3235
Epoch 0 Batch 60: Train Loss = 0.3648
Epoch 0 Batch 80: Train Loss = 0.2472
Epoch 0 Batch 100: Train Loss = 0.2660
Epoch 0 Batch 120: Train Loss = 0.2637


/home/ubicomp/choczhang/EMR/challenge/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])


Epoch 0: Loss = 0.3021 Valid loss = 0.2574 roc = 0.7276
confusion matrix:
[[3918    0]
 [ 308    0]]
accuracy = 0.9271178245544434
precision class 0 = 0.9271178245544434
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.7276282293510472
AUC of PRC = 0.2362470339138863
min(+P, Se) = 0.32142857142857145
f1_score = nan
------------ Save best model - AUROC: 0.7276 ------------

------------ Save best-prc model ------------


------------ Save best-sum model ------------

Epoch 1 Batch 0: Train Loss = 0.2744
Epoch 1 Batch 20: Train Loss = 0.2491
Epoch 1 Batch 40: Train Loss = 0.2580
Epoch 1 Batch 60: Train Loss = 0.2182
Epoch 1 Batch 80: Train Loss = 0.2777
Epoch 1 Batch 100: Train Loss = 0.2307
Epoch 1 Batch 120: Train Loss = 0.2535
Epoch 1: Loss = 0.2649 Valid loss = 0.2420 roc = 0.7467
confusion matrix:
[[3918    0]
 [ 308    0]]
accuracy = 0.9271178245544434
precision class 0 = 0.9271178245544434
precision class 1 = nan
recall class 0 = 1.0
recall class 1 

/home/ubicomp/choczhang/EMR/challenge/utils/metrics.py:31: RuntimeWarning: invalid value encountered in double_scalars
  f1_score=2*prec1*rec1/(prec1+rec1)


...

Epoch 108 Batch 0: Train Loss = 0.0567
Epoch 108 Batch 20: Train Loss = 0.0997
Epoch 108 Batch 40: Train Loss = 0.1184
Epoch 108 Batch 60: Train Loss = 0.0649
Epoch 108 Batch 80: Train Loss = 0.0800
Epoch 108 Batch 100: Train Loss = 0.0525
Epoch 108 Batch 120: Train Loss = 0.1391
Epoch 108: Loss = 0.0964 Valid loss = 0.1332 roc = 0.9463
confusion matrix:
[[3880   38]
 [ 123  185]]
accuracy = 0.9619024991989136
precision class 0 = 0.969273030757904
precision class 1 = 0.8295964002609253
recall class 0 = 0.9903011918067932
recall class 1 = 0.600649356842041
AUC of ROC = 0.9462910111838136
AUC of PRC = 0.7629561591525194
min(+P, Se) = 0.6881028938906752
f1_score = 0.6967984932387027
Epoch 109 Batch 0: Train Loss = 0.1249
Epoch 109 Batch 20: Train Loss = 0.0985
Epoch 109 Batch 40: Train Loss = 0.0807
Epoch 109 Batch 60: Train Loss = 0.0698
Epoch 109 Batch 80: Train Loss = 0.0850
Epoch 109 Batch 100: Train Loss = 0.1154
Epoch 109 Batch 120: Train Loss = 0.1658
Epoch 109: Loss = 0.0971 Valid

Epoch 120 Batch 0: Train Loss = 0.1197
Epoch 120 Batch 20: Train Loss = 0.1184
Epoch 120 Batch 40: Train Loss = 0.1350
Epoch 120 Batch 60: Train Loss = 0.1022
Epoch 120 Batch 80: Train Loss = 0.1206
Epoch 120 Batch 100: Train Loss = 0.0757
Epoch 120 Batch 120: Train Loss = 0.0833
Epoch 120: Loss = 0.1002 Valid loss = 0.1374 roc = 0.9450
confusion matrix:
[[3884   34]
 [ 138  170]]
accuracy = 0.9592995643615723
precision class 0 = 0.9656887054443359
precision class 1 = 0.8333333134651184
recall class 0 = 0.9913221001625061
recall class 1 = 0.551948070526123
AUC of ROC = 0.9449518704878583
AUC of PRC = 0.7476225172882403
min(+P, Se) = 0.6796116504854369
f1_score = 0.664062535710401
Epoch 121 Batch 0: Train Loss = 0.0899
Epoch 121 Batch 20: Train Loss = 0.1211
Epoch 121 Batch 40: Train Loss = 0.0944
Epoch 121 Batch 60: Train Loss = 0.1788
Epoch 121 Batch 80: Train Loss = 0.0944
Epoch 121 Batch 100: Train Loss = 0.0580
Epoch 121 Batch 120: Train Loss = 0.0390
Epoch 121: Loss = 0.0950 Valid

Epoch 132 Batch 0: Train Loss = 0.0949
Epoch 132 Batch 20: Train Loss = 0.0607
Epoch 132 Batch 40: Train Loss = 0.0977
Epoch 132 Batch 60: Train Loss = 0.0746
Epoch 132 Batch 80: Train Loss = 0.1537
Epoch 132 Batch 100: Train Loss = 0.0526
Epoch 132 Batch 120: Train Loss = 0.0536
Epoch 132: Loss = 0.0901 Valid loss = 0.1302 roc = 0.9485
confusion matrix:
[[3871   47]
 [ 125  183]]
accuracy = 0.9592995643615723
precision class 0 = 0.9687187075614929
precision class 1 = 0.7956521511077881
recall class 0 = 0.9880040884017944
recall class 1 = 0.5941558480262756
AUC of ROC = 0.9484704295194342
AUC of PRC = 0.7678346029290138
min(+P, Se) = 0.6948051948051948
f1_score = 0.6802974211464726
Epoch 133 Batch 0: Train Loss = 0.0682
Epoch 133 Batch 20: Train Loss = 0.1279
Epoch 133 Batch 40: Train Loss = 0.1150
Epoch 133 Batch 60: Train Loss = 0.0698
Epoch 133 Batch 80: Train Loss = 0.1197
Epoch 133 Batch 100: Train Loss = 0.0703
Epoch 133 Batch 120: Train Loss = 0.0818
Epoch 133: Loss = 0.0928 Val

Epoch 144 Batch 0: Train Loss = 0.0664
Epoch 144 Batch 20: Train Loss = 0.1037
Epoch 144 Batch 40: Train Loss = 0.1076
Epoch 144 Batch 60: Train Loss = 0.1036
Epoch 144 Batch 80: Train Loss = 0.0641
Epoch 144 Batch 100: Train Loss = 0.0626
Epoch 144 Batch 120: Train Loss = 0.0614
Epoch 144: Loss = 0.0891 Valid loss = 0.1373 roc = 0.9432
confusion matrix:
[[3880   38]
 [ 128  180]]
accuracy = 0.9607193470001221
precision class 0 = 0.9680638909339905
precision class 1 = 0.8256880640983582
recall class 0 = 0.9903011918067932
recall class 1 = 0.5844155550003052
AUC of ROC = 0.9431627586298337
AUC of PRC = 0.7629678867951573
min(+P, Se) = 0.6818181818181818
f1_score = 0.6844106519528524
Epoch 145 Batch 0: Train Loss = 0.0862
Epoch 145 Batch 20: Train Loss = 0.1405
Epoch 145 Batch 40: Train Loss = 0.0887
Epoch 145 Batch 60: Train Loss = 0.0864
Epoch 145 Batch 80: Train Loss = 0.0529
Epoch 145 Batch 100: Train Loss = 0.0636
Epoch 145 Batch 120: Train Loss = 0.0680
Epoch 145: Loss = 0.0853 Val

NameError: name 'best_minpse' is not defined

In [18]:
checkpoint = torch.load(file_name+'')
save_epoch = checkpoint['epoch']
print("last saved model is in epoch {}".format(save_epoch))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()


last saved model is in epoch 89


vanilla_transformer_encoder(
  (PositionalEncoding): PositionalEncoding(
    (dropout): Dropout(p=0)
  )
  (GRUs): ModuleList(
    (0): GRU(1, 32, batch_first=True)
    (1): GRU(1, 32, batch_first=True)
    (2): GRU(1, 32, batch_first=True)
    (3): GRU(1, 32, batch_first=True)
    (4): GRU(1, 32, batch_first=True)
    (5): GRU(1, 32, batch_first=True)
    (6): GRU(1, 32, batch_first=True)
    (7): GRU(1, 32, batch_first=True)
    (8): GRU(1, 32, batch_first=True)
    (9): GRU(1, 32, batch_first=True)
    (10): GRU(1, 32, batch_first=True)
    (11): GRU(1, 32, batch_first=True)
    (12): GRU(1, 32, batch_first=True)
    (13): GRU(1, 32, batch_first=True)
    (14): GRU(1, 32, batch_first=True)
    (15): GRU(1, 32, batch_first=True)
    (16): GRU(1, 32, batch_first=True)
    (17): GRU(1, 32, batch_first=True)
    (18): GRU(1, 32, batch_first=True)
    (19): GRU(1, 32, batch_first=True)
    (20): GRU(1, 32, batch_first=True)
    (21): GRU(1, 32, batch_first=True)
    (22): GRU(1, 32, batc

In [19]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(test_x, test_y, test_mask_x, test_x_len, batch_size, shuffle=True)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt= model(batch_x, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Batch 0: Test Loss = 0.1431

==>Predicting on test
Test Loss = 0.1122
confusion matrix:
[[3701   40]
 [ 112  182]]
accuracy = 0.962329626083374
precision class 0 = 0.9706268310546875
precision class 1 = 0.8198198080062866
recall class 0 = 0.9893076419830322
recall class 1 = 0.6190476417541504
AUC of ROC = 0.9558141353306894
AUC of PRC = 0.7838274764292477
min(+P, Se) = 0.711864406779661
f1_score = 0.7054263669584423
